
Using EarthquakeDataset.txt dataset;

Please find the list of aftershocks (within 50 km and in 5 days) for
top 5 earthquakes and order them according to aftershock count.

In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version
!pip install pyspark

'apt-get' is not recognized as an internal or external command,
operable program or batch file.
Sistem belirtilen yolu bulam�yor.
'update-alternatives' is not recognized as an internal or external command,
operable program or batch file.
java version "17.0.6" 2023-01-17 LTS
Java(TM) SE Runtime Environment (build 17.0.6+9-LTS-190)
Java HotSpot(TM) 64-Bit Server VM (build 17.0.6+9-LTS-190, mixed mode, sharing)


In [ ]:
from pyspark import SparkContext

# import file
sc = SparkContext.getOrCreate()
eqRDD = sc.textFile("EarthquakeDataset.txt")
eqRDD = eqRDD.filter(lambda x : 'Enlem' not in x)

# function to formatting data
def data_parse(line):
    info = line.split("\t")
    day = info[2]
    latitude = float(info[4])
    longitude = float(info[5])
    mag = float(info[7])
    loc = info[14].strip()
    char_remov = ["(", ")", "[", "]"]
    for char in char_remov:
      loc = loc.replace(char, "")
    return (day,(latitude,longitude),mag, loc[0:30])
# take data 
eqRDD = eqRDD.map(lambda line : data_parse(line))
eqRDD = eqRDD.sortBy(lambda x : x[2],ascending=False)
eqRDD.take(5)

In [231]:
#create list of 5 days.
dayRDD = eqRDD.map(lambda line : line[0])
top_5 = dayRDD.take(5)
top_5

['1939.12.26', '2023.02.06', '1926.06.26', '2023.02.06', '1930.05.06']

In [232]:
from datetime import datetime, timedelta
# create a list for the days
def calculate5_days(day_list):
    updated_day_list = []
    for day in day_list:
        date_format = "%Y.%m.%d"
        dayf = datetime.strptime(day, date_format)
        updated_day = dayf + timedelta(days=5)
        consecutive_dates = [(dayf + timedelta(days=i)).strftime(date_format) for i in range(0, 6)]
        updated_day_list.extend(consecutive_dates)
    return updated_day_list

days_list = calculate5_days(top_5)
# days_list

aftershocks_rdd = eqRDD.filter(lambda x: x[0] in days_list)
aftershocks_rdd.take(10)

[('1939.12.26', (39.8, 39.51), 7.9, 'KURUTILEK- ERZINCAN North East'),
 ('2023.02.06', (37.1757, 37.085), 7.7, 'YAMACOBA-SEHITKAMIL GAZIANTEP '),
 ('1926.06.26', (36.54, 27.33), 7.7, 'ONIKI ADALAR AKDENIZ'),
 ('2023.02.06', (38.0818, 37.1773), 7.6, 'EKINOZU KAHRAMANMARAS North We'),
 ('1930.05.06', (37.98, 44.48), 7.6, 'T�RKIYE-IRAN SINIR B�LGESI'),
 ('2023.02.06', (37.2337, 36.7805), 6.6, 'BELPINAR-NURDAGI GAZIANTEP Nor'),
 ('2023.02.06', (38.031, 36.335), 6.0, 'SOGUKPINAR-GOKSUN KAHRAMANMARA'),
 ('2023.02.06', (38.1855, 37.9793), 5.8, 'SUCATI-DOGANSEHIR MALATYA Sout'),
 ('2023.02.06', (38.1433, 37.7937), 5.8, 'CAVUSLU-DOGANSEHIR MALATYA Sou'),
 ('1939.12.28', (40.47, 37.0), 5.8, 'KOKLUCE-NIKSAR TOKAT South Eas')]

In [233]:
import geopy.distance
# function to calculate distance between earthquake and aftershocks
def get_distances(tuple1, tuple2):
    eqday = tuple1[0]
    asday = tuple2[0]
    eqcoords = tuple1[1]
    ascoords = tuple2[1]
    loc1 = tuple1[3]
    loc2 = tuple2[3]
    distance = geopy.distance.geodesic(eqcoords, ascoords).km
    distance = round(distance,2)
    return ((eqday,loc1),(asday, loc2),distance, 1)

# create rdd from top 5 earthquake
top_list= eqRDD.take(5)
topRDD= sc.parallelize(top_list)

# create combinations of biggest eq's and aftershocks using cartesian function and filter distances high than 50 km
combinationRDD = topRDD.cartesian(aftershocks_rdd)
combinationRDD = combinationRDD.map(lambda x: get_distances(x[0],x[1]))
combinationRDD = combinationRDD.filter(lambda x: x[2]<= 50 and x[2] != 0)
combinationRDD.take(5)

[(('2023.02.06', 'YAMACOBA-SEHITKAMIL GAZIANTEP '),
  ('2023.02.06', 'BELPINAR-NURDAGI GAZIANTEP Nor'),
  27.79,
  1),
 (('2023.02.06', 'YAMACOBA-SEHITKAMIL GAZIANTEP '),
  ('2023.02.06', 'KIRISKAL-NURDAGI GAZIANTEP Nor'),
  23.56,
  1),
 (('2023.02.06', 'YAMACOBA-SEHITKAMIL GAZIANTEP '),
  ('2023.02.06', 'KARAHUYUK-PAZARCIK KAHRAMANMAR'),
  9.54,
  1),
 (('2023.02.06', 'YAMACOBA-SEHITKAMIL GAZIANTEP '),
  ('2023.02.06', 'IKIZKUYU-NURDAGI GAZIANTEP Nor'),
  10.93,
  1),
 (('1939.12.26', 'KURUTILEK- ERZINCAN North East'),
  ('1939.12.29', 'AGILOZU- ERZINCAN West 3.7 km'),
  22.84,
  1)]

In [234]:
combinationRDD = combinationRDD.map(lambda x:(x[0],x[3]))
totalRDD=combinationRDD.reduceByKey(lambda x,y:x+y).sortBy(lambda x: x[1], ascending=False)

totalRDD.take(10)

[(('2023.02.06', 'EKINOZU KAHRAMANMARAS North We'), 185),
 (('2023.02.06', 'YAMACOBA-SEHITKAMIL GAZIANTEP '), 131),
 (('1930.05.06', 'T�RKIYE-IRAN SINIR B�LGESI'), 7),
 (('1926.06.26', 'ONIKI ADALAR AKDENIZ'), 2),
 (('1939.12.26', 'KURUTILEK- ERZINCAN North East'), 2)]